## Agentic Retrieval in Azure AI Search and Azure AI Agent Service


### 1. Load Connections

In [1]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import os

load_dotenv(override=True) # take environment variables from .env.

# The following variables from your .env file are used in this notebook
project_conn_str = os.environ["PROJECT_CONNECTION_STRING"]
agent_model = os.getenv("AGENT_MODEL", "gpt-4o")
endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://search.azure.com/.default")
index_name = os.getenv("AZURE_SEARCH_INDEX", "earth_at_night")
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_gpt_deployment = os.getenv("AZURE_OPENAI_GPT_DEPLOYMENT", "gpt-4o")
azure_openai_gpt_model = os.getenv("AZURE_OPENAI_GPT_MODEL", "gpt-4o")
azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
azure_openai_embedding_model = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL", "text-embedding-3-large")
agent_name = os.getenv("AZURE_SEARCH_AGENT_NAME", "earth-search-agent")
api_version = "2025-05-01-Preview"

### 2. Create Search Index

In [2]:
from azure.search.documents.indexes.models import SearchIndex, SearchField, VectorSearch, VectorSearchProfile, HnswAlgorithmConfiguration, AzureOpenAIVectorizer, AzureOpenAIVectorizerParameters, SemanticSearch, SemanticConfiguration, SemanticPrioritizedFields, SemanticField
from azure.search.documents.indexes import SearchIndexClient

index = SearchIndex(
    name=index_name,
    fields=[
        SearchField(name="id", type="Edm.String", key=True, filterable=True, sortable=True, facetable=True),
        SearchField(name="page_chunk", type="Edm.String", filterable=False, sortable=False, facetable=False),
        SearchField(name="page_embedding_text_3_large", type="Collection(Edm.Single)", stored=False, vector_search_dimensions=3072, vector_search_profile_name="hnsw_text_3_large"),
        SearchField(name="page_number", type="Edm.Int32", filterable=True, sortable=True, facetable=True),
        SearchField(name="chapter_number", type="Edm.Int32", filterable=True, sortable=True, facetable=True)
    ],
    vector_search=VectorSearch(
        profiles=[VectorSearchProfile(name="hnsw_text_3_large", algorithm_configuration_name="alg", vectorizer_name="azure_openai_text_3_large")],
        algorithms=[HnswAlgorithmConfiguration(name="alg")],
        vectorizers=[
            AzureOpenAIVectorizer(
                vectorizer_name="azure_openai_text_3_large",
                parameters=AzureOpenAIVectorizerParameters(
                    resource_url=azure_openai_endpoint,
                    deployment_name=azure_openai_embedding_deployment,
                    model_name=azure_openai_embedding_model
                )
            )
        ]
    ),
    semantic_search=SemanticSearch(
        default_configuration_name="semantic_config",
        configurations=[
            SemanticConfiguration(
                name="semantic_config",
                prioritized_fields=SemanticPrioritizedFields(
                    content_fields=[
                        SemanticField(field_name="page_chunk")
                    ]
                )
            )
        ]
    )
)

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.create_or_update_index(index)
print(f"Index '{index_name}' created or updated successfully")


Index 'earth_at_night' created or updated successfully


### 3. Upload sample documents

In [3]:
import requests
from azure.search.documents import SearchIndexingBufferedSender
from azure.search.documents import SearchClient

url = "https://raw.githubusercontent.com/Azure-Samples/azure-search-sample-data/refs/heads/main/nasa-e-book/earth-at-night-json/documents.json"
documents = requests.get(url).json()

with SearchIndexingBufferedSender(endpoint=endpoint, index_name=index_name, credential=credential) as client:
    client.upload_documents(documents=documents)

print(f"Documents uploaded to index '{index_name}'")


Documents uploaded to index 'earth_at_night'


### 4. Create search agent

In [4]:
import requests

create_agent_request = {
    "name": agent_name,
    "targetIndexes": [ { "indexName": index_name } ],
    "models": [
          {
            "kind": "azureOpenAI",
            "azureOpenAIParameters": {
                "resourceUri": azure_openai_endpoint,
                "apiKey": None,
                "deploymentId": azure_openai_gpt_model,
                "modelName": azure_openai_gpt_model
            }
        }
    ]
}

response = requests.put(
    url=f"{endpoint}/agents/{agent_name}?api-version={api_version}",
    headers={ "Authorization": f"Bearer {token_provider()}" },
    json=create_agent_request
)
response.raise_for_status()


### 5. Create AI Agent

In [10]:
from azure.ai.projects import AIProjectClient
import json
project_client = AIProjectClient.from_connection_string(project_conn_str, credential=credential)

instructions = """
An Q&A agent that can answer questions about the Earth at night.
Sources have a JSON format with a ref_id that must be cited in the answer.
If you do not have the answer, respond with "I don't know".
"""
agent = project_client.agents.create_agent(
    model=agent_model,
    name=agent_name,
    instructions=instructions
)
print(json.dumps(agent.as_dict(), indent=2))

{
  "id": "asst_tybwnQVSaJkQOuKHkFSll4Bb",
  "object": "assistant",
  "created_at": 1746141946,
  "name": "earth-search-agent",
  "description": null,
  "model": "gpt-4o",
  "instructions": "\nAn Q&A agent that can answer questions about the Earth at night.\nSources have a JSON format with a ref_id that must be cited in the answer.\nIf you do not have the answer, respond with \"I don't know\".\n",
  "tools": [],
  "top_p": 1.0,
  "temperature": 1.0,
  "tool_resources": {},
  "metadata": {},
  "response_format": "auto"
}


### 6. Add Agentic Retrieval tool to AI Agent

In [11]:
from azure.ai.projects.models import FunctionTool, ToolSet, ListSortOrder

thread = project_client.agents.create_thread()
retrieval_results = {}

def agentic_retrieval() -> str:
    """
        Searches a NASA e-book about images of Earth at night and other science related facts.
        The returned string is in a JSON format that contains the reference id.
        Be sure to use the same format in your agent's response
    """
    # Take the last 5 messages in the conversation
    messages = project_client.agents.list_messages(thread.id, limit=5, order=ListSortOrder.DESCENDING)
    # Reverse the order so the most recent message is last
    messages.data.reverse()
    retrieval_request = {
        "messages" : [ { "role": msg.role, "content": [ { "text": msg_content.text.value, "type": msg_content.type } for msg_content in msg.content ] } for msg in messages.data ],
        "targetIndexParams" :  [
            { 
                "indexName" : index_name,
                "rerankerThreshold": 2.5
            } 
        ]
    }
    response = requests.post(
        url=f"{endpoint}/agents/{agent_name}/retrieve?api-version={api_version}",
        headers={ "Authorization": f"Bearer {token_provider()}" },
        json=retrieval_request
    )
    response.raise_for_status()
    result = response.json()

    # Associate the retrieval results with the last message in the conversation
    last_message = messages.data[-1]
    retrieval_results[last_message.id] = result

    # Return the grounding response to the agent
    return result["response"][0]["content"][0]["text"]

# Link to AI Agent Service function calling tutorial
user_functions = { agentic_retrieval }
functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)
project_client.agents.enable_auto_function_calls(toolset=toolset)

### 7. Start a chat with the agent

In [ ]:
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="""
        Why do suburban belts display larger December brightening than urban cores even though absolute light levels are higher downtown?
        Why is the Phoenix nighttime street grid is so sharply visible from space, whereas large stretches of the interstate between midwestern cities remain comparatively dim?
    """
)

run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id, toolset=toolset)
if run.status == "failed":
    raise RuntimeError(f"Run failed: {run.last_error}")
output = project_client.agents.list_messages(thread_id=thread.id).get_last_text_message_by_role("assistant").text.value

print("Agent response:", output.replace(".", "\n"))


### 7.1: Review retrieval activity and results

In [ ]:
retrieval_results = retrieval_results.get(message.id)
if retrieval_results is None:
    raise RuntimeError(f"No retrieval results found for message {message.id}")

print("Retrieval activity")
print(json.dumps(retrieval_results["activity"], indent=2))
print("Retrieval results")
print(json.dumps(retrieval_results["references"], indent=2))

Retrieval activity
[
  {
    "type": "ModelQueryPlanning",
    "id": 0,
    "inputTokens": 1359,
    "outputTokens": 516
  },
  {
    "type": "AzureSearchQuery",
    "id": 1,
    "targetIndex": "earth_at_night",
    "query": {
      "search": "suburban belts December brightening compared to urban cores",
      "filter": null
    },
    "queryTime": "2025-04-30T03:45:03.614Z",
    "elapsedMs": 437
  },
  {
    "type": "AzureSearchQuery",
    "id": 2,
    "targetIndex": "earth_at_night",
    "query": {
      "search": "Phoenix nighttime street grid visibility from space",
      "filter": null
    },
    "queryTime": "2025-04-30T03:45:03.957Z",
    "count": 2,
    "elapsedMs": 342
  },
  {
    "type": "AzureSearchQuery",
    "id": 3,
    "targetIndex": "earth_at_night",
    "query": {
      "search": "dim interstates between midwestern cities compared to Phoenix",
      "filter": null
    },
    "queryTime": "2025-04-30T03:45:04.250Z",
    "elapsedMs": 293
  }
]
Retrieval results
[
  {
  

### 8. Continue the conversation

In [ ]:
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="How do I find lava at night?"
)

run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id, toolset=toolset)
if run.status == "failed":
    raise RuntimeError(f"Run failed: {run.last_error}")

output = project_client.agents.list_messages(thread_id=thread.id).get_last_text_message_by_role("assistant").text.value
print("Agent response:", output.replace(".", "\n"))


Agent response: To find lava at night, you can use the following methods:

1
 **Infrared Imaging:**
   - Satellite instruments such as the Thermal Infrared Sensor (TIRS) on Landsat 8 can detect the thermal (infrared) signature of lava
 This allows for the observation of both hot and cooling lava flows which are not visible in regular imagery ([ref: 2, 3, 4])

   - Combining thermal imaging with nightlight data provides a comprehensive view of volcanic activity, distinguishing the hot lava from the cooler surroundings


2
 **Visible Light Monitoring:**
   - The Visible Infrared Imaging Radiometer Suite (VIIRS) Day/Night Band (DNB) on polar-orbiting satellites can detect faint sources of illumination such as moonlight, which can highlight glowing lava flows at night ([ref: 4, 5])

   - The DNB can capture the glow from active vents and lava flows, making it possible to track volcanic activity even in the absence of sunlight


3
 **Astronaut Photography:**
   - Nighttime photographs taken

### 8.1: Review retrieval activity and results

In [ ]:
retrieval_results = retrieval_results.get(message.id)
if retrieval_results is None:
    raise RuntimeError(f"No retrieval results found for message {message.id}")

print("Retrieval activity")
print(json.dumps(retrieval_results["activity"], indent=2))
print("Retrieval results")
print(json.dumps(retrieval_results["references"], indent=2))

Retrieval activity
[
  {
    "type": "ModelQueryPlanning",
    "id": 0,
    "inputTokens": 1816,
    "outputTokens": 291
  },
  {
    "type": "AzureSearchQuery",
    "id": 1,
    "targetIndex": "earth_at_night",
    "query": {
      "search": "locating lava flows at night",
      "filter": null
    },
    "queryTime": "2025-04-30T03:45:39.901Z",
    "count": 6,
    "elapsedMs": 19627
  },
  {
    "type": "AzureSearchQuery",
    "id": 2,
    "targetIndex": "earth_at_night",
    "query": {
      "search": "nighttime visibility of lava",
      "filter": null
    },
    "queryTime": "2025-04-30T03:45:40.301Z",
    "count": 6,
    "elapsedMs": 400
  },
  {
    "type": "AzureSearchQuery",
    "id": 3,
    "targetIndex": "earth_at_night",
    "query": {
      "search": "equipment for finding lava at night",
      "filter": null
    },
    "queryTime": "2025-04-30T03:45:40.584Z",
    "elapsedMs": 282
  }
]
Retrieval results
[
  {
    "type": "AzureSearchDoc",
    "id": "0",
    "activitySource